# ETAPA 5: Análise das Variáveis de Múltipla Escolha

## Consulta à Comunidade UFPE sobre o Uso de Inteligência Artificial

---

**Objetivo:** Identificar prioridades e preferências da comunidade.

**Variáveis analisadas:**
- Contextos de uso de IA (7 opções predefinidas)
- Temas prioritários para o Plano Institucional (8 opções)
- Tipos de formação/suporte necessários (5 opções)

**Procedimentos:**
1. Separar respostas múltiplas (delimiter: vírgula ou ponto e vírgula) em categorias individuais
2. Calcular frequência de menção de cada categoria
3. Ordenar categorias por frequência (ranking)
4. Estratificar por vínculo para identificar diferenças de prioridades
5. Gerar visualizações (gráficos de barras horizontais, nuvens de palavras)

**Produto:** Rankings de prioridades; tabelas estratificadas; visualizações comparativas.

**Critério de Validação:** Cobertura de todas as categorias predefinidas; identificação clara das top 3-5 prioridades.

---

## Requisitos Técnicos

### Kernel Python
- **Python:** 3.9 ou superior
- **Kernel recomendado:** `python3` ou ambiente virtual com as dependências instaladas

### Dados de Entrada

| Arquivo | Descrição | Origem |
|---------|-----------|--------|
| `dados_limpos_etapa1.xlsx` | Base de dados limpa com 2.164 registros | Etapa 1 |

### Dados de Saída

| Arquivo | Descrição |
|---------|----------|
| `multipla_escolha_etapa5.xlsx` | Tabelas de frequência e rankings |
| `grafico_contextos_uso_etapa5.png` | Ranking dos contextos de uso de IA |
| `grafico_temas_prioritarios_etapa5.png` | Ranking dos temas prioritários |
| `grafico_tipos_formacao_etapa5.png` | Ranking dos tipos de formação |
| `grafico_comparativo_vinculo_etapa5.png` | Comparação entre vínculos |
| `relatorio_multipla_escolha_etapa5.md` | Relatório com rankings e análises |

---

## 1. Instalação de Dependências

In [ ]:
# ============================================================================
# INSTALAÇÃO DE DEPENDÊNCIAS
# Execute esta célula apenas uma vez para instalar os pacotes necessários
# ============================================================================

!pip install pandas openpyxl matplotlib seaborn numpy wordcloud --quiet

print("✅ Dependências instaladas com sucesso!")

## 2. Configuração Inicial

In [ ]:
# ============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Tentar importar wordcloud (opcional)
try:
    from wordcloud import WordCloud
    WORDCLOUD_DISPONIVEL = True
except ImportError:
    WORDCLOUD_DISPONIVEL = False
    print("⚠️ Biblioteca 'wordcloud' não disponível. Nuvens de palavras serão omitidas.")

# Configurações de exibição
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

# Configurações de visualização
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10
sns.set_style("whitegrid")

# Verificar versões
print("Bibliotecas carregadas com sucesso!")
print(f"  • Pandas: {pd.__version__}")
print(f"  • Matplotlib: {plt.matplotlib.__version__}")
print(f"  • Seaborn: {sns.__version__}")
print(f"  • WordCloud: {'Disponível' if WORDCLOUD_DISPONIVEL else 'Não disponível'}")

In [ ]:
# ============================================================================
# CONFIGURAÇÃO DE ARQUIVOS DE ENTRADA E SAÍDA
# ============================================================================

# Arquivo de entrada (gerado na Etapa 1)
ARQUIVO_ENTRADA = "data/dados_limpos_etapa1.xlsx"

# Arquivos de saída
ARQUIVO_SAIDA_EXCEL = "data/multipla_escolha_etapa5.xlsx"
ARQUIVO_SAIDA_RELATORIO = "data/relatorio_multipla_escolha_etapa5.md"

print("Configuração de arquivos:")
print(f"  📥 Entrada: {ARQUIVO_ENTRADA}")
print(f"  📤 Saídas:  {ARQUIVO_SAIDA_EXCEL}")
print(f"            {ARQUIVO_SAIDA_RELATORIO}")

## 3. Carregamento dos Dados

In [ ]:
# Carregar dados limpos da Etapa 1
try:
    df = pd.read_excel(ARQUIVO_ENTRADA)
    print(f"✅ Dados carregados com sucesso!")
    print(f"   Registros: {len(df)}")
    print(f"   Variáveis: {len(df.columns)}")
except FileNotFoundError:
    print(f"❌ ERRO: Arquivo '{ARQUIVO_ENTRADA}' não encontrado!")
    print(f"   Certifique-se de que a Etapa 1 foi executada e o arquivo está no diretório correto.")
    raise

In [ ]:
# ============================================================================
# IDENTIFICAÇÃO DAS VARIÁVEIS DE MÚLTIPLA ESCOLHA
# ============================================================================

# Dicionário com as variáveis de múltipla escolha e suas descrições
variaveis_multipla = {
    'Contextos_Uso': {
        'descricao': 'Em quais contextos você utiliza ferramentas de IA?',
        'nome_curto': 'Contextos de Uso'
    },
    'Temas_Prioritarios': {
        'descricao': 'Quais temas devem ser prioritários no Plano Institucional de IA?',
        'nome_curto': 'Temas Prioritários'
    },
    'Tipos_Formacao': {
        'descricao': 'Quais tipos de formação/suporte são necessários?',
        'nome_curto': 'Tipos de Formação'
    }
}

print("=" * 80)
print("VARIÁVEIS DE MÚLTIPLA ESCOLHA PARA ANÁLISE")
print("=" * 80)

for var, info in variaveis_multipla.items():
    n_respostas = df[var].notna().sum()
    print(f"\n📌 {var}")
    print(f"   Descrição: {info['descricao']}")
    print(f"   Respostas válidas: {n_respostas} ({n_respostas/len(df)*100:.1f}%)")
    
    # Mostrar exemplo de resposta
    exemplo = df[var].dropna().iloc[0] if n_respostas > 0 else 'N/A'
    print(f"   Exemplo: {exemplo[:100]}..." if len(str(exemplo)) > 100 else f"   Exemplo: {exemplo}")

---

## 4. Funções Auxiliares

In [ ]:
# ============================================================================
# FUNÇÕES AUXILIARES PARA ANÁLISE DE MÚLTIPLA ESCOLHA
# ============================================================================

def separar_respostas_multiplas(serie, delimitadores=[';', ',']):
    """
    Separa respostas múltiplas em itens individuais.
    Retorna uma lista com todos os itens mencionados.
    """
    todas_respostas = []
    
    for resposta in serie.dropna():
        resposta_str = str(resposta)
        
        # Tentar separar por cada delimitador
        itens = [resposta_str]
        for delim in delimitadores:
            novos_itens = []
            for item in itens:
                novos_itens.extend(item.split(delim))
            itens = novos_itens
        
        # Limpar e adicionar
        for item in itens:
            item_limpo = item.strip()
            if item_limpo and item_limpo.lower() not in ['nan', 'none', '']:
                todas_respostas.append(item_limpo)
    
    return todas_respostas


def calcular_frequencia_mencoes(serie, n_respondentes=None):
    """
    Calcula frequência de menção de cada categoria.
    Retorna DataFrame com ranking.
    """
    todas_respostas = separar_respostas_multiplas(serie)
    contagem = Counter(todas_respostas)
    
    # Criar DataFrame
    df_freq = pd.DataFrame(contagem.items(), columns=['Categoria', 'Menções'])
    df_freq = df_freq.sort_values('Menções', ascending=False).reset_index(drop=True)
    
    # Calcular percentuais
    total_mencoes = df_freq['Menções'].sum()
    n_resp = n_respondentes or serie.notna().sum()
    
    df_freq['% Menções'] = (df_freq['Menções'] / total_mencoes * 100).round(1)
    df_freq['% Respondentes'] = (df_freq['Menções'] / n_resp * 100).round(1)
    df_freq['Ranking'] = range(1, len(df_freq) + 1)
    
    # Reordenar colunas
    df_freq = df_freq[['Ranking', 'Categoria', 'Menções', '% Menções', '% Respondentes']]
    
    return df_freq, total_mencoes, n_resp


def calcular_frequencia_por_vinculo(df, variavel, variavel_vinculo='Vinculo_Padronizado'):
    """
    Calcula frequência de menção estratificada por vínculo.
    Retorna DataFrame com percentuais por grupo.
    """
    resultados = {}
    
    for vinculo in df[variavel_vinculo].dropna().unique():
        df_vinculo = df[df[variavel_vinculo] == vinculo]
        respostas = separar_respostas_multiplas(df_vinculo[variavel])
        contagem = Counter(respostas)
        n_resp = df_vinculo[variavel].notna().sum()
        
        # Calcular percentual sobre respondentes do grupo
        for categoria, mencoes in contagem.items():
            if categoria not in resultados:
                resultados[categoria] = {}
            resultados[categoria][vinculo] = round(mencoes / n_resp * 100, 1)
    
    # Criar DataFrame
    df_vinculo = pd.DataFrame(resultados).T
    
    # Ordenar por total de menções
    df_vinculo['Total'] = df_vinculo.sum(axis=1)
    df_vinculo = df_vinculo.sort_values('Total', ascending=False)
    df_vinculo = df_vinculo.drop('Total', axis=1)
    
    return df_vinculo


def criar_grafico_ranking(df_freq, titulo, arquivo_saida, top_n=None, cor='steelblue'):
    """
    Cria gráfico de barras horizontais com ranking.
    """
    dados = df_freq.head(top_n) if top_n else df_freq
    
    fig, ax = plt.subplots(figsize=(12, max(6, len(dados) * 0.4)))
    
    # Criar gradiente de cores
    cores = plt.cm.Blues(np.linspace(0.3, 0.9, len(dados)))[::-1]
    
    # Plotar barras
    y_pos = range(len(dados))
    bars = ax.barh(y_pos, dados['% Respondentes'], color=cores, edgecolor='white')
    
    # Configurar eixo y
    ax.set_yticks(y_pos)
    ax.set_yticklabels(dados['Categoria'])
    ax.invert_yaxis()
    
    # Labels nas barras
    for bar, mencoes, pct in zip(bars, dados['Menções'], dados['% Respondentes']):
        ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
                f'{pct}% ({mencoes})', va='center', fontsize=9)
    
    ax.set_xlabel('% dos Respondentes')
    ax.set_title(titulo, fontsize=14, fontweight='bold')
    
    # Ajustar limite x
    ax.set_xlim(0, dados['% Respondentes'].max() * 1.2)
    
    plt.tight_layout()
    plt.savefig(arquivo_saida, dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\nGráfico salvo como: {arquivo_saida}")


print("✅ Funções auxiliares carregadas com sucesso!")

---

## 5. Análise: Contextos de Uso de IA

In [ ]:
# ============================================================================
# ANÁLISE DOS CONTEXTOS DE USO
# ============================================================================

print("=" * 80)
print("CONTEXTOS DE USO DE FERRAMENTAS DE IA")
print("=" * 80)

# Calcular frequências
freq_contextos, total_mencoes_ctx, n_resp_ctx = calcular_frequencia_mencoes(df['Contextos_Uso'])

print(f"\nTotal de respondentes: {n_resp_ctx}")
print(f"Total de menções: {total_mencoes_ctx}")
print(f"Média de contextos por respondente: {total_mencoes_ctx/n_resp_ctx:.1f}")
print(f"\nRanking de contextos de uso:\n")
print(freq_contextos.to_string(index=False))

In [ ]:
# Gráfico de ranking - Contextos de Uso
criar_grafico_ranking(
    freq_contextos,
    'Contextos de Uso de Ferramentas de IA',
    'grafico_contextos_uso_etapa5.png'
)

In [ ]:
# Estratificação por vínculo - Contextos de Uso
freq_contextos_vinculo = calcular_frequencia_por_vinculo(df, 'Contextos_Uso')

print("\n" + "=" * 100)
print("CONTEXTOS DE USO POR VÍNCULO INSTITUCIONAL (% de cada grupo)")
print("=" * 100)
print(freq_contextos_vinculo.round(1))

---

## 6. Análise: Temas Prioritários

In [ ]:
# ============================================================================
# ANÁLISE DOS TEMAS PRIORITÁRIOS
# ============================================================================

print("=" * 80)
print("TEMAS PRIORITÁRIOS PARA O PLANO INSTITUCIONAL DE IA")
print("=" * 80)

# Calcular frequências
freq_temas, total_mencoes_temas, n_resp_temas = calcular_frequencia_mencoes(df['Temas_Prioritarios'])

print(f"\nTotal de respondentes: {n_resp_temas}")
print(f"Total de menções: {total_mencoes_temas}")
print(f"Média de temas por respondente: {total_mencoes_temas/n_resp_temas:.1f}")
print(f"\nRanking de temas prioritários:\n")
print(freq_temas.to_string(index=False))

In [ ]:
# Gráfico de ranking - Temas Prioritários
criar_grafico_ranking(
    freq_temas,
    'Temas Prioritários para o Plano Institucional de IA',
    'grafico_temas_prioritarios_etapa5.png'
)

In [ ]:
# Estratificação por vínculo - Temas Prioritários
freq_temas_vinculo = calcular_frequencia_por_vinculo(df, 'Temas_Prioritarios')

print("\n" + "=" * 100)
print("TEMAS PRIORITÁRIOS POR VÍNCULO INSTITUCIONAL (% de cada grupo)")
print("=" * 100)
print(freq_temas_vinculo.round(1))

---

## 7. Análise: Tipos de Formação/Suporte

In [ ]:
# ============================================================================
# ANÁLISE DOS TIPOS DE FORMAÇÃO/SUPORTE
# ============================================================================

print("=" * 80)
print("TIPOS DE FORMAÇÃO/SUPORTE NECESSÁRIOS")
print("=" * 80)

# Calcular frequências
freq_formacao, total_mencoes_form, n_resp_form = calcular_frequencia_mencoes(df['Tipos_Formacao'])

print(f"\nTotal de respondentes: {n_resp_form}")
print(f"Total de menções: {total_mencoes_form}")
print(f"Média de tipos por respondente: {total_mencoes_form/n_resp_form:.1f}")
print(f"\nRanking de tipos de formação:\n")
print(freq_formacao.to_string(index=False))

In [ ]:
# Gráfico de ranking - Tipos de Formação
criar_grafico_ranking(
    freq_formacao,
    'Tipos de Formação/Suporte Necessários',
    'grafico_tipos_formacao_etapa5.png'
)

In [ ]:
# Estratificação por vínculo - Tipos de Formação
freq_formacao_vinculo = calcular_frequencia_por_vinculo(df, 'Tipos_Formacao')

print("\n" + "=" * 100)
print("TIPOS DE FORMAÇÃO POR VÍNCULO INSTITUCIONAL (% de cada grupo)")
print("=" * 100)
print(freq_formacao_vinculo.round(1))

---

## 8. Visualizações Comparativas

In [ ]:
# ============================================================================
# GRÁFICO COMPARATIVO POR VÍNCULO - TOP 5 DE CADA VARIÁVEL
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 8))

# Dados para plotar (top 5 de cada)
dados_plot = [
    (freq_contextos_vinculo.head(5), 'Contextos de Uso\n(Top 5)'),
    (freq_temas_vinculo.head(5), 'Temas Prioritários\n(Top 5)'),
    (freq_formacao_vinculo.head(5), 'Tipos de Formação\n(Top 5)')
]

for idx, (dados, titulo) in enumerate(dados_plot):
    ax = axes[idx]
    
    # Plotar heatmap
    sns.heatmap(dados, annot=True, fmt='.0f', cmap='YlGnBu', 
                cbar_kws={'label': '% do grupo'}, ax=ax,
                linewidths=0.5, linecolor='white')
    
    ax.set_title(titulo, fontsize=12, fontweight='bold')
    ax.set_xlabel('Vínculo Institucional')
    ax.set_ylabel('')
    
    # Rotacionar labels do eixo x
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=9)
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=9)

plt.suptitle('Comparação das Prioridades por Vínculo Institucional', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('grafico_comparativo_vinculo_etapa5.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_comparativo_vinculo_etapa5.png")

In [ ]:
# ============================================================================
# PAINEL CONSOLIDADO - TOP 5 DE CADA VARIÁVEL
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Dados para plotar
dados_barras = [
    (freq_contextos.head(5), 'Contextos de Uso', 'Blues'),
    (freq_temas.head(5), 'Temas Prioritários', 'Greens'),
    (freq_formacao.head(5), 'Tipos de Formação', 'Oranges')
]

for idx, (dados, titulo, cmap) in enumerate(dados_barras):
    ax = axes[idx]
    
    cores = plt.cm.get_cmap(cmap)(np.linspace(0.3, 0.8, len(dados)))[::-1]
    
    bars = ax.barh(range(len(dados)), dados['% Respondentes'], color=cores)
    
    ax.set_yticks(range(len(dados)))
    ax.set_yticklabels(dados['Categoria'], fontsize=9)
    ax.invert_yaxis()
    ax.set_xlabel('% dos Respondentes')
    ax.set_title(f'Top 5 - {titulo}', fontsize=11, fontweight='bold')
    
    # Labels nas barras
    for bar, pct in zip(bars, dados['% Respondentes']):
        ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
                f'{pct}%', va='center', fontsize=9)
    
    ax.set_xlim(0, dados['% Respondentes'].max() * 1.15)

plt.suptitle('Ranking das Prioridades da Comunidade UFPE', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('grafico_painel_ranking_etapa5.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_painel_ranking_etapa5.png")

In [ ]:
# ============================================================================
# NUVENS DE PALAVRAS (se biblioteca disponível)
# ============================================================================

if WORDCLOUD_DISPONIVEL:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    variaveis_wc = [
        ('Contextos_Uso', 'Contextos de Uso', 'Blues'),
        ('Temas_Prioritarios', 'Temas Prioritários', 'Greens'),
        ('Tipos_Formacao', 'Tipos de Formação', 'Oranges')
    ]
    
    for idx, (var, titulo, colormap) in enumerate(variaveis_wc):
        ax = axes[idx]
        
        # Obter todas as respostas
        respostas = separar_respostas_multiplas(df[var])
        texto = ' '.join(respostas)
        
        # Gerar nuvem de palavras
        wordcloud = WordCloud(
            width=800, height=400,
            background_color='white',
            colormap=colormap,
            max_words=50,
            min_font_size=10
        ).generate(texto)
        
        ax.imshow(wordcloud, interpolation='bilinear')
        ax.axis('off')
        ax.set_title(titulo, fontsize=12, fontweight='bold')
    
    plt.suptitle('Nuvens de Palavras - Prioridades da Comunidade', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('grafico_nuvens_palavras_etapa5.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\nGráfico salvo como: grafico_nuvens_palavras_etapa5.png")
else:
    print("⚠️ Nuvens de palavras não geradas (biblioteca 'wordcloud' não disponível).")

---

## 9. Análise de Diferenças entre Vínculos

In [ ]:
# ============================================================================
# IDENTIFICAÇÃO DE DIFERENÇAS NOTÁVEIS ENTRE VÍNCULOS
# ============================================================================

def identificar_diferencas(df_vinculo, limiar=10):
    """
    Identifica categorias com diferenças notáveis entre vínculos.
    Limiar: diferença mínima em pontos percentuais.
    """
    diferencas = []
    
    for categoria in df_vinculo.index:
        valores = df_vinculo.loc[categoria].dropna()
        if len(valores) > 1:
            diff = valores.max() - valores.min()
            if diff >= limiar:
                diferencas.append({
                    'Categoria': categoria,
                    'Maior': f"{valores.idxmax()} ({valores.max():.1f}%)",
                    'Menor': f"{valores.idxmin()} ({valores.min():.1f}%)",
                    'Diferença (pp)': round(diff, 1)
                })
    
    return pd.DataFrame(diferencas).sort_values('Diferença (pp)', ascending=False)


print("=" * 90)
print("DIFERENÇAS NOTÁVEIS ENTRE VÍNCULOS (≥ 10 pontos percentuais)")
print("=" * 90)

print("\n📌 CONTEXTOS DE USO")
diff_contextos = identificar_diferencas(freq_contextos_vinculo)
if len(diff_contextos) > 0:
    print(diff_contextos.to_string(index=False))
else:
    print("   Nenhuma diferença ≥ 10pp identificada.")

print("\n📌 TEMAS PRIORITÁRIOS")
diff_temas = identificar_diferencas(freq_temas_vinculo)
if len(diff_temas) > 0:
    print(diff_temas.to_string(index=False))
else:
    print("   Nenhuma diferença ≥ 10pp identificada.")

print("\n📌 TIPOS DE FORMAÇÃO")
diff_formacao = identificar_diferencas(freq_formacao_vinculo)
if len(diff_formacao) > 0:
    print(diff_formacao.to_string(index=False))
else:
    print("   Nenhuma diferença ≥ 10pp identificada.")

---

## 10. Resumo Executivo

In [ ]:
print("=" * 80)
print("RESUMO EXECUTIVO - ETAPA 5")
print("=" * 80)

print(f"""
📊 PRINCIPAIS ACHADOS

1. CONTEXTOS DE USO DE IA (Top 5)
""")
for _, row in freq_contextos.head(5).iterrows():
    print(f"   {row['Ranking']}. {row['Categoria']}: {row['% Respondentes']}%")

print(f"""
2. TEMAS PRIORITÁRIOS PARA O PLANO INSTITUCIONAL (Top 5)
""")
for _, row in freq_temas.head(5).iterrows():
    print(f"   {row['Ranking']}. {row['Categoria']}: {row['% Respondentes']}%")

print(f"""
3. TIPOS DE FORMAÇÃO/SUPORTE NECESSÁRIOS (Top 5)
""")
for _, row in freq_formacao.head(5).iterrows():
    print(f"   {row['Ranking']}. {row['Categoria']}: {row['% Respondentes']}%")

print(f"""
📌 ESTATÍSTICAS GERAIS

   • Contextos de uso: {len(freq_contextos)} categorias identificadas
   • Temas prioritários: {len(freq_temas)} categorias identificadas
   • Tipos de formação: {len(freq_formacao)} categorias identificadas

📌 DIFERENÇAS ENTRE VÍNCULOS

   • Contextos de uso: {len(diff_contextos)} diferenças notáveis (≥10pp)
   • Temas prioritários: {len(diff_temas)} diferenças notáveis (≥10pp)
   • Tipos de formação: {len(diff_formacao)} diferenças notáveis (≥10pp)
""")

---

## 11. Salvar Resultados

In [ ]:
# Salvar todas as tabelas em um arquivo Excel com múltiplas abas
with pd.ExcelWriter(ARQUIVO_SAIDA_EXCEL, engine='openpyxl') as writer:
    # Rankings gerais
    freq_contextos.to_excel(writer, sheet_name='Ranking_Contextos', index=False)
    freq_temas.to_excel(writer, sheet_name='Ranking_Temas', index=False)
    freq_formacao.to_excel(writer, sheet_name='Ranking_Formacao', index=False)
    
    # Estratificação por vínculo
    freq_contextos_vinculo.to_excel(writer, sheet_name='Contextos_por_Vinculo')
    freq_temas_vinculo.to_excel(writer, sheet_name='Temas_por_Vinculo')
    freq_formacao_vinculo.to_excel(writer, sheet_name='Formacao_por_Vinculo')
    
    # Diferenças notáveis
    if len(diff_contextos) > 0:
        diff_contextos.to_excel(writer, sheet_name='Diff_Contextos', index=False)
    if len(diff_temas) > 0:
        diff_temas.to_excel(writer, sheet_name='Diff_Temas', index=False)
    if len(diff_formacao) > 0:
        diff_formacao.to_excel(writer, sheet_name='Diff_Formacao', index=False)

print(f"✅ Tabelas salvas em: {ARQUIVO_SAIDA_EXCEL}")

In [ ]:
# Gerar relatório em Markdown
relatorio = f"""# RELATÓRIO DE ANÁLISE DAS VARIÁVEIS DE MÚLTIPLA ESCOLHA
## Etapa 5 — Consulta à Comunidade UFPE sobre o Uso de IA

**Data de execução:** {datetime.now().strftime("%d/%m/%Y às %H:%M")}

---

## 1. CONTEXTOS DE USO DE FERRAMENTAS DE IA

**Respondentes:** {n_resp_ctx} | **Total de menções:** {total_mencoes_ctx} | **Média por respondente:** {total_mencoes_ctx/n_resp_ctx:.1f}

### Ranking Completo

| # | Categoria | Menções | % Respondentes |
|---|-----------|---------|----------------|
"""

for _, row in freq_contextos.iterrows():
    relatorio += f"| {row['Ranking']} | {row['Categoria']} | {row['Menções']} | {row['% Respondentes']}% |\n"

relatorio += f"""
---

## 2. TEMAS PRIORITÁRIOS PARA O PLANO INSTITUCIONAL

**Respondentes:** {n_resp_temas} | **Total de menções:** {total_mencoes_temas} | **Média por respondente:** {total_mencoes_temas/n_resp_temas:.1f}

### Ranking Completo

| # | Categoria | Menções | % Respondentes |
|---|-----------|---------|----------------|
"""

for _, row in freq_temas.iterrows():
    relatorio += f"| {row['Ranking']} | {row['Categoria']} | {row['Menções']} | {row['% Respondentes']}% |\n"

relatorio += f"""
---

## 3. TIPOS DE FORMAÇÃO/SUPORTE NECESSÁRIOS

**Respondentes:** {n_resp_form} | **Total de menções:** {total_mencoes_form} | **Média por respondente:** {total_mencoes_form/n_resp_form:.1f}

### Ranking Completo

| # | Categoria | Menções | % Respondentes |
|---|-----------|---------|----------------|
"""

for _, row in freq_formacao.iterrows():
    relatorio += f"| {row['Ranking']} | {row['Categoria']} | {row['Menções']} | {row['% Respondentes']}% |\n"

relatorio += f"""
---

## 4. DIFERENÇAS NOTÁVEIS ENTRE VÍNCULOS (≥ 10pp)

### Contextos de Uso
"""

if len(diff_contextos) > 0:
    relatorio += "\n| Categoria | Maior | Menor | Diferença |\n|-----------|-------|-------|-----------|\n"
    for _, row in diff_contextos.iterrows():
        relatorio += f"| {row['Categoria']} | {row['Maior']} | {row['Menor']} | {row['Diferença (pp)']}pp |\n"
else:
    relatorio += "\nNenhuma diferença ≥ 10pp identificada.\n"

relatorio += "\n### Temas Prioritários\n"

if len(diff_temas) > 0:
    relatorio += "\n| Categoria | Maior | Menor | Diferença |\n|-----------|-------|-------|-----------|\n"
    for _, row in diff_temas.iterrows():
        relatorio += f"| {row['Categoria']} | {row['Maior']} | {row['Menor']} | {row['Diferença (pp)']}pp |\n"
else:
    relatorio += "\nNenhuma diferença ≥ 10pp identificada.\n"

relatorio += "\n### Tipos de Formação\n"

if len(diff_formacao) > 0:
    relatorio += "\n| Categoria | Maior | Menor | Diferença |\n|-----------|-------|-------|-----------|\n"
    for _, row in diff_formacao.iterrows():
        relatorio += f"| {row['Categoria']} | {row['Maior']} | {row['Menor']} | {row['Diferença (pp)']}pp |\n"
else:
    relatorio += "\nNenhuma diferença ≥ 10pp identificada.\n"

relatorio += f"""
---

## 5. PRINCIPAIS CONCLUSÕES

1. **Contexto mais frequente:** "{freq_contextos.iloc[0]['Categoria']}" ({freq_contextos.iloc[0]['% Respondentes']}% dos respondentes).

2. **Tema prioritário principal:** "{freq_temas.iloc[0]['Categoria']}" ({freq_temas.iloc[0]['% Respondentes']}% dos respondentes).

3. **Tipo de formação mais demandado:** "{freq_formacao.iloc[0]['Categoria']}" ({freq_formacao.iloc[0]['% Respondentes']}% dos respondentes).

4. **Consenso entre grupos:** A comunidade UFPE apresenta {'alto grau de alinhamento' if (len(diff_contextos) + len(diff_temas) + len(diff_formacao)) < 5 else 'algumas divergências'} nas prioridades entre diferentes vínculos institucionais.

---

## 6. ARQUIVOS GERADOS

| Arquivo | Descrição |
|---------|----------|
| `{ARQUIVO_SAIDA_EXCEL}` | Tabelas de frequência e rankings |
| `grafico_contextos_uso_etapa5.png` | Ranking dos contextos de uso |
| `grafico_temas_prioritarios_etapa5.png` | Ranking dos temas prioritários |
| `grafico_tipos_formacao_etapa5.png` | Ranking dos tipos de formação |
| `grafico_comparativo_vinculo_etapa5.png` | Heatmaps comparativos por vínculo |
| `grafico_painel_ranking_etapa5.png` | Painel consolidado Top 5 |
| `grafico_nuvens_palavras_etapa5.png` | Nuvens de palavras (se disponível) |

---

*Relatório gerado automaticamente em {datetime.now().strftime("%d/%m/%Y às %H:%M")}*
"""

# Salvar relatório
with open(ARQUIVO_SAIDA_RELATORIO, "w", encoding='utf-8') as f:
    f.write(relatorio)

print(f"✅ Relatório salvo em: {ARQUIVO_SAIDA_RELATORIO}")

In [ ]:
print("\n" + "=" * 70)
print("ETAPA 5 CONCLUÍDA COM SUCESSO!")
print("=" * 70)
print(f"""
📁 ARQUIVOS GERADOS:

   📊 Dados:
      • {ARQUIVO_SAIDA_EXCEL}

   📈 Gráficos:
      • grafico_contextos_uso_etapa5.png
      • grafico_temas_prioritarios_etapa5.png
      • grafico_tipos_formacao_etapa5.png
      • grafico_comparativo_vinculo_etapa5.png
      • grafico_painel_ranking_etapa5.png
      • grafico_nuvens_palavras_etapa5.png (se wordcloud disponível)

   📝 Relatório:
      • {ARQUIVO_SAIDA_RELATORIO}

▶️  PRÓXIMA ETAPA: Etapa 6 - Análise Qualitativa das Respostas Abertas
""")

---

## Resumo dos Arquivos

### Entrada
| Arquivo | Origem |
|---------|--------|
| `dados_limpos_etapa1.xlsx` | Etapa 1 |

### Saída
| Arquivo | Descrição |
|---------|----------|
| `multipla_escolha_etapa5.xlsx` | Tabelas de frequência e rankings (até 9 abas) |
| `grafico_contextos_uso_etapa5.png` | Ranking dos contextos de uso de IA |
| `grafico_temas_prioritarios_etapa5.png` | Ranking dos temas prioritários |
| `grafico_tipos_formacao_etapa5.png` | Ranking dos tipos de formação |
| `grafico_comparativo_vinculo_etapa5.png` | Heatmaps comparativos por vínculo |
| `grafico_painel_ranking_etapa5.png` | Painel consolidado com Top 5 |
| `grafico_nuvens_palavras_etapa5.png` | Nuvens de palavras (opcional) |
| `relatorio_multipla_escolha_etapa5.md` | Relatório completo com rankings |

---

**Próxima etapa:** Etapa 6 - Análise Qualitativa das Respostas Abertas